### Earthquake Predictor

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
data = pd.read_csv("Earthquake.csv")
data.columns


Index(['Date', 'Time', 'Latitude', 'Longitude', 'Type', 'Depth', 'Depth Error',
       'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status'],
      dtype='object')

In [12]:
data = data[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]
data.head()

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,02-01-1965,13.44.18,19.246,145.616,131.6,6.0
1,04-01-1965,11.29.49,1.863,127.352,80.0,5.8
2,05-01-1965,18.05.58,-20.579,-173.972,20.0,6.2
3,08-01-1965,18.49.43,-59.076,-23.557,15.0,5.8
4,09-01-1965,13.32.50,11.938,126.427,15.0,5.8


In [13]:
import datetime
import time

timestamp = []
for d, t in zip(data['Date'], data['Time']):
    try:
        ts = datetime.datetime.strptime(d+' '+t, '%m-%d-%Y %H.%M.%S')
        timestamp.append(time.mktime(ts.timetuple()))
    except:
        # print('ValueError')
        timestamp.append('ValueError')
timeStamp = pd.Series(timestamp)
data['Timestamp'] = timeStamp.values
final_data = data.drop(['Date', 'Time'], axis=1)
final_data = final_data[final_data.Timestamp != 'ValueError']
final_data.head()

,Latitude,Longitude,Depth,Magnitude,Timestamp
1456,-29.400,-177.169,35.0,5.6,42060.0
1457,24.185,102.543,11.3,7.1,7817441.0
1458,23.984,102.732,15.0,5.9,10390750.0
1459,-9.583,151.493,15.0,6.3,13046754.0
1460,15.785,-59.808,36.7,6.0,15647174.0


In [14]:
X = final_data[['Timestamp', 'Latitude', 'Longitude']]
y = final_data[['Magnitude', 'Depth']]


In [15]:
X = np.asarray(X).astype('float32')
y = np.asarray(y).astype('float32')


In [25]:
X

array([[ 4.2060000e+04, -2.9400000e+01, -1.7716901e+02],
       [ 7.8174410e+06,  2.4184999e+01,  1.0254300e+02],
       [ 1.0390750e+07,  2.3983999e+01,  1.0273200e+02],
       ...,
       [ 1.4762697e+09, -5.6592999e+00,  1.5447340e+02],
       [ 1.4789414e+09, -9.1237001e+00, -1.0984920e+02],
       [ 1.4789518e+09, -1.0964000e+01,  1.6157230e+02]], dtype=float32)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

(6899, 3) (1725, 3) (6899, 2) (1725, 3)


In [18]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(neurons, activation, optimizer, loss):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [19]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=0)

# neurons = [16, 64, 128, 256]
neurons = [16]
# batch_size = [10, 20, 50, 100]
batch_size = [10]
epochs = [10]
# activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'exponential']
activation = ['sigmoid', 'relu']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizer = ['SGD', 'Adadelta']
loss = ['squared_hinge']

param_grid = dict(neurons=neurons, batch_size=batch_size, epochs=epochs, activation=activation, optimizer=optimizer, loss=loss)

In [21]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.600000 using {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.600000 (0.489898) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.400000 (0.489898) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}
0.400000 (0.489898) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.600000 (0.489898) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}


In [32]:
model = Sequential()
model.add(Dense(16, activation='sigmoid', input_shape=(3,)))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='SGD', loss='squared_hinge', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=10, epochs=10, verbose=1, validation_data=(X_test, y_test))

[test_loss, test_acc] = model.evaluate(X_test, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

Epoch 1/10
690/690 [==============================] - 2s 2ms/step - loss: 0.0026 - accuracy: 0.9841 - val_loss: 0.0049 - val_accuracy: 0.9797
Epoch 2/10
690/690 [==============================] - 1s 2ms/step - loss: 0.0026 - accuracy: 0.9841 - val_loss: 0.0049 - val_accuracy: 0.9797
Epoch 3/10
690/690 [==============================] - 1s 2ms/step - loss: 0.0026 - accuracy: 0.9841 - val_loss: 0.0049 - val_accuracy: 0.9797
Epoch 4/10
690/690 [==============================] - 1s 2ms/step - loss: 0.0026 - accuracy: 0.9841 - val_loss: 0.0049 - val_accuracy: 0.9797
Epoch 5/10
690/690 [==============================] - 1s 2ms/step - loss: 0.0026 - accuracy: 0.9841 - val_loss: 0.0049 - val_accuracy: 0.9797
Epoch 6/10
690/690 [==============================] - 1s 2ms/step - loss: 0.0026 - accuracy: 0.9841 - val_loss: 0.0049 - val_accuracy: 0.9797
Epoch 7/10
690/690 [==============================] - 1s 2ms/step - loss: 0.0026 - accuracy: 0.9841 - val_loss: 0.0049 - val_accuracy: 0.9797
Epoch 

In [33]:
model.predict(X_test)

array([[0.18178806, 0.8182119 ],
       [0.18178806, 0.8182119 ],
       [0.18178806, 0.8182119 ],
       ...,
       [0.18178806, 0.8182119 ],
       [0.18178806, 0.8182119 ],
       [0.18178806, 0.8182119 ]], dtype=float32)

In [34]:
sample = [[-29.400, -177.169, 42060.0]]

model.predict(sample)

array([[0.15623249, 0.84376746]], dtype=float32)

In [35]:
model.save("Earthquake")
model.save_weights("weights.h5")



INFO:tensorflow:Assets written to: Earthquake\assets
